# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [5]:
%load_ext autoreload
%autoreload 2
from TelechargementDonnees import CreationDriverFirefox, JourneeSiteNba, PasDeMatchError
import os
import pandas as pd
import numpy as np
from copy import deepcopy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [4]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2021-01-01')
except PasDeMatchError as e : 
    print(e)
#exporter
journee.saveCsv()

In [ ]:
# pour un ensemeble de journee : 
for j in [f'2020-12-{d}' for d in range(27,32)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j)
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

## Telecharger les données liees aux joueurs depuis le site internet 
1. on se connecte à la page principale des joueurs
1. on passe de lettre en lettre pour sauvegarder les donnees

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException

In [26]:
from TelechargementDonnees import CreationDriverFirefox,urlSiteNbaJoueur
import time

In [11]:
#creer driver et aller sur la page des joueurs
driver=CreationDriverFirefox()
driver.get(urlSiteNbaJoueur)

In [23]:
#atteindre l'entite parent des initiales
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
containerBouttonLettre=WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(EC.element_to_be_clickable((
                    By.XPATH, "//div[@class='hidden-sm col-sm-9 col-lg-10 letters-wrap']")))

In [24]:
#liste des bouttons lettres
listBouttonLettre=WebDriverWait(containerBouttonLettre, 10,ignored_exceptions=ignored_exceptions).until(EC.presence_of_all_elements_located((
                    By.XPATH, ".//*")))

In [33]:
#creer la df des Joueurs
dico={}
for i,e in enumerate(listBouttonLettre) : 
    e.click()
    time.sleep(5)
    dico[i]=pd.read_html(driver.page_source)
dfJoueurs=pd.concat([v[0] for v in dico.values()])

In [36]:
#mettre en forme la df
dfJoueursForme=dfJoueurs.drop('Unnamed: 1',axis=1).rename(
    columns={'Joueur':'nom','Équipe':'equipe','POS':'position','Taille':'taille',
             'OUEST':'poids', 'EXP':'experience','Pays':'pays'})
dfJoueursForme['poids']=dfJoueursForme.poids.apply(lambda x : float(x[:-3]))
dfJoueursForme['date_entree_nba']=dfJoueursForme['experience'].apply(lambda x : pd.to_datetime('2020-10-01') - 
                                  pd.to_timedelta(x*365.25, unit='D'))

In [42]:
dfJoueursForme['experience'].unique()

array([0, 2, 7, 3, 14, 1, 10, 6, 17, 16, 12, 8, 4, 9, 5, 13, 11, 15],
      dtype=object)

In [50]:
dfJoueursForme['date_entree_nba']=dfJoueursForme['experience'].apply(lambda x : pd.to_datetime('2020-10-01') - 
                                  pd.to_timedelta(x*365.25, unit='D'))